In [1]:
import re
import time
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import scipy
#Chi-square test
from scipy.stats import chi2_contingency, chi2
#ANOVA test
import statsmodels.api as sm
from statsmodels.formula.api import ols

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

import math
import warnings
warnings.filterwarnings('ignore')

### Cleaning data

In [2]:
#Column: Noi_dung
def noi_dung_clean(feature,df):
    df[feature] = df[feature].str.lower()
    index_lst=[]
    for e in ['bán đất', 'villa', 'biệt thự', 'trọ ', 'căn hộ dịch vụ', 'khách sạn']:
        for i in range(0, df.shape[0]):
            if re.findall(e, df.Noi_dung[i]) != []:
                index_lst.append(i)
    df = df.drop(index_lst).reset_index(drop=True)
    return df

In [3]:
#Column: Dien_tich, Dien_tich_su_dung
def dien_tich_clean(feature_lst, df):
    for feature in feature_lst:
        df[feature] = df[feature].apply(lambda x: x if (type(x)==float) | (type(x)==int) \
                                              else float(re.sub("\.", "",x,1)))
        df[feature] = df[feature].astype(float)
    return df

In [4]:
#Column: Gia
def gia_clean(feature,df):
    #Loại bỏ khoảng trắng 2 đầu, lowercase, split into list of strings
    df[feature] = df[feature].apply(lambda x: x.strip().lower().split(' '))
    #Loại bỏ dòng 'đã bán'
    sold_index = []
    for i in range(0, df.shape[0]):
        if df[feature][i][0]=='đã':
            sold_index.append(i)
    df = df.drop(sold_index).reset_index(drop=True)
    
    for i in range(0, df.shape[0]):
        #Đơn vị của tổng tiền là 'tỷ' => nhân 1000 để chuyển sang triệu
        if df[feature][i][1] =='tỷ' or df[feature][i][1] =='tỉ':
            df[feature][i] = float(df[feature][i][0].replace(',','.'))*1000
        #Đơn vị của tổng tiền là 'triệu'
        elif (df[feature][i][1] =='triệu') and (float(df['Gia'][i][0].replace(',','.')) >=500):
            df[feature][i] = float(df[feature][i][0].replace(',','.'))
        else:
            df = df.drop(i)
    df[feature] = df[feature].astype(float)
    df.reset_index(drop=True, inplace=True)
    return df

In [5]:
#Column: Cau_truc
def cau_truc_clean(feature,df):
    for i in range(0, df.shape[0]):
        count = 0
        df[feature][i] = df[feature][i].replace(',',' ').replace('+', ' ').lower().split()
        for e in df[feature][i]:
            if e in ['hầm', 'hâm', 'trệt']:
                count+=1
            if e in ['lửng', 'lung', 'lưng','thượng', 'thuong']:
                count+=0.5
            if e in ['áp', 'gác', 'ap', 'gac']:
                count+=0.5
            if e in ['lầu', 'lâu', 'lau']:
                count+=int(df[feature][i][df[feature][i].index(e)-1])   
        df[feature][i] = float(count)
    df[feature] = df[feature].astype(float)
    return df

In [6]:
#Column: Tien_ich
def count_tien_ich(lst_loai_tien_ich,lst_tien_ich):
    count=0
    for tien_ich in lst_loai_tien_ich:
        if tien_ich in lst_tien_ich:
            count+=1
    return count

def tien_ich_clean(feature, df):
    df[feature] = df[feature].fillna('0')
    df[feature] = df[feature].apply(lambda x: x.lower())
    lst_loai_tien_ich = ['chợ','siêu thị','bệnh viện','công viên','trung tâm','trường học', 'để xe']
    df[feature] = df[feature].apply(lambda x: count_tien_ich(lst_loai_tien_ich,x))
    return df

In [7]:
#Column: So_phong, Nha_ve_sinh
def so_phong_nvs_clean(feature, df):
    mode = df[feature].mode()
    for i in range(df.shape[0]):
        if isinstance(df[feature][i], int)==False:
            df[feature][i] = mode
    df[feature] = df[feature].astype(int)
    return df

In [8]:
#Column: Quan, Phuong
def phuong_quan_combine(feature1, feature2, new_feature, df):
    df[feature1] = df[feature1].apply(lambda x: x.strip())
    df[feature2] = df[feature2].apply(lambda x: x.strip())
    df[new_feature] = df[feature1] + ', ' + df[feature2]
    return df

In [9]:
#Column: Giay_to
def giay_to_clean(feature, df):
    df[feature] = df[feature].apply(lambda x: 1 if x=='Sổ hồng' or x =='Sổ đỏ' else 0)
    return df

In [10]:
#Column: Huong
def huong_clean(feature, df):
    huong_dict = {'Đ.Bắc':1, 'T.Nam':2, 'Bắc':3, 'Đ.Nam':4, 'T.Bắc':5, 'Đông':6, 'Nam':7, 'Tây':8,'Không xác định':0}
    df[feature] = [huong_dict[i] for i in df[feature]]
    return df

In [11]:
#Column: Hem_rong & Duong_mat_tien
def hem_duong(feature1,feature2,new_feature,df):
    for i in range(df.shape[0]):
        if (df[feature1][i] !=0) & (df[feature2][i] !=0):
            df[new_feature][i] = df[feature2][i]
        else:
            df[new_feature] = df[feature1] + df[feature2]
    return df

In [30]:
def load_n_clean_data_train(train_data_file):
    df = pd.read_excel(train_data_file, engine='openpyxl')
    df1 = noi_dung_clean('Noi_dung',df)
    df2 = gia_clean('Gia',df1)
    df3 = dien_tich_clean(['Dien_tich','Dien_tich_su_dung'], df2)
    df4 = cau_truc_clean('Cau_truc',df3)
    df5 = tien_ich_clean('Tien_ich', df4)
    df6 = so_phong_nvs_clean('So_phong', df5)
    df7 = so_phong_nvs_clean('Nha_ve_sinh', df6)
    df8 = phuong_quan_combine('Quan', 'Phuong','Dia_diem', df7)
    df9 = giay_to_clean('Giay_to', df8)
    df10 = huong_clean('Huong', df9)
    df11 = hem_duong('Hem_rong','Duong_mat_tien','Hem_duong',df10)
#     df12 =  drop_outliers(['So_phong','Dien_tich', 'Dien_tich_su_dung','Hem_duong'], df11)
    df11 = df11.reset_index(drop=True)
    return df11[['Gia','Nha_ve_sinh','Cau_truc', 'Dien_tich','Dien_tich_su_dung','Hem_duong',  
                   'Duong','Dia_diem']]

In [13]:
#Column: Gia - Test data
def gia_clean_test(feature,df):
    df['Gia'] = df[feature]*1000
    return df

In [14]:
def load_n_clean_data_test(test_data_file):
    df1 = pd.read_excel(test_data_file, engine='openpyxl')
    df2 = gia_clean_test('Gia (tỷ - dùng để đánh giá )',df1)
    df3 = dien_tich_clean(['Dien_tich','Dien_tich_su_dung'], df2)
    df4 = cau_truc_clean('Cau_truc',df3)
    df5 = tien_ich_clean('Tien_ich', df4)
    df6 = so_phong_nvs_clean('So_phong', df5)
    df7 = so_phong_nvs_clean('Nha_ve_sinh', df6)
    df8 = phuong_quan_combine('Quan', 'Phuong','Dia_diem', df7)
    df9 = giay_to_clean('Giay_to', df8)
    df10 = huong_clean('Huong', df9)
    df11 = hem_duong('Hem_rong','Duong_mat_tien','Hem_duong',df10)
    return df11[['Gia','Nha_ve_sinh','Cau_truc', 'Dien_tich','Dien_tich_su_dung','Hem_duong',  
                   'Duong','Dia_diem']]

### Univariate  analysis

#### --Number variable 

In [15]:
def univariate_cont_analysis(var,df):
    mean = df[var].mean()
    median = df[var].median() 
    mode = df[var].mode()
    max_val = df[var].max()
    min_val = df[var].min()
    range_val = np.ptp(df[var])
    variance = df[var].var()
    skewness = df[var].skew()
    kurtosis = df[var].kurtosis()
    result = [mean,median,mode,max_val,min_val,range_val,variance,skewness,kurtosis]
    return result

In [16]:
#Visualization - Histogram
def visualize_histogram(cont_var, df):
    if len(cont_var)%3 == 0:
        n = len(cont_var)/3
    else:
        n = len(cont_var)//3 + 1
    plt.figure(figsize=(15,15))
    for i in range(1,len(cont_var)+1):
        plt.subplot(n,3,i)
        sns.distplot(df[cont_var[i-1]].dropna())
    plt.tight_layout()
    plt.show()

In [17]:
#Visualization - Boxplot
def visualize_boxplot(cont_var, df):
    if len(cont_var)%3 == 0:
        n = len(cont_var)/3
    else:
        n = len(cont_var)//3 + 1
    plt.figure(figsize=(15,15))
    for i in range (1,len(cont_var)+1):
        plt.subplot(n,3,i)
        plt.boxplot(df[cont_var[i-1]].dropna())
        plt.title(cont_var[i-1])
    plt.tight_layout()
    plt.show()

#### --Catergorical variable

In [18]:
#Visualization - Value_count - barplot
def visualize_cat_value_count(cat_var, df):
    if len(cat_var)%3 == 0:
        n = len(cat_var)/3
    else:
        n = len(cat_var)//3 + 1
    plt.figure(figsize=(15,6))
    for i in range (1,len(cat_var)+1):
        count = df.groupby(df[cat_var[i-1]].dropna()).size()
        #print(count)
        plt.subplot(n,3,i)
        count.plot.bar()
    plt.tight_layout()
    plt.show()

### Bivariate  analysis

In [19]:
##continuous - continuous 
def visualize_heatmap(cont_var, df):
    plt.figure(figsize=(10,10))
    sns.heatmap(df[cont_var].corr(), cmap="YlGnBu", annot=True,square=True, linewidths=.5)
    plt.show()

In [20]:
# categorical - output(cont) 
def visualize_boxplot_cat_output(cat_var, output, df):
    if len(cat_var)%3 == 0:
        n = len(cat_var)/3
    else:
        n = len(cat_var)//3 + 1
    plt.figure(figsize=(15,5))
    for i in range(1,len(cat_var)+1):
        df_melt = df[[cat_var[i-1],output]]
        plt.subplot(n,3,i)
        sns.boxplot(data=df_melt,x=cat_var[i-1],y=output )
        i+=1
    plt.tight_layout()
    plt.show()

In [21]:
# categorical - output(cont) 
# Anova Test

def anova_test(cat_var, output, df):
    dependent_cat_list = []
    for var in cat_var:
        model = ols(' '+ output +'~ C(' + var + ')', df).fit()
        anova_table = sm.stats.anova_lm(model, typ=2)
        #display(anova_table)
        if anova_table['PR(>F)'][0] < 0.05:
            #print(var,'p-value:',anova_table['PR(>F)'][0])
            #print('==> Dependent (Reject H0)')
            dependent_cat_list.append(var)
    return dependent_cat_list

In [22]:
#categgorical - categorical
def chi2_test(cat_var,df):
    chi2_test_df=pd.DataFrame(index=cat_var,columns=cat_var)
    alpha = 0.05

    for i in range (len(cat_var)):
        for j in range(i+1,len(cat_var)):
            table = pd.crosstab(df[cat_var[i]],df[cat_var[j]])
            stat, p, dof, expected = chi2_contingency(table)
            #print('\n*** significance = %.3f, p_value = %.3f' %(alpha,p))
            if p < alpha:
                #print(cat_var[i],'&',cat_var[j],'==> Dependent (Reject H0)')
                chi2_test_df.loc[cat_var[i],cat_var[j]]='Dependent'
            else:
                #print(cat_var[i],'&',cat_var[j],'==> Independent (Fail to Reject H0)')
                chi2_test_df.loc[cat_var[i],cat_var[j]]='Independent'
    return chi2_test_df

### Outliers

In [23]:
def drop_outliers(cont_vars, df):
    for cont_var in cont_vars:
        upper_limit= np.quantile(df[cont_var],0.75) + 1.5*scipy.stats.iqr(df[cont_var])
        lower_limit= np.quantile(df[cont_var],0.25) - 1.5*scipy.stats.iqr(df[cont_var])
        df = df[ (df[cont_var]>=lower_limit) & (df[cont_var]<=upper_limit)]
    return df

In [24]:
def number_of_outliers(cont_vars,df):
    df_outliers = pd.DataFrame(index=cont_vars, columns=['upper_outliers','lower_outliers','outlier_per'])
    for cont_var in cont_vars:
        upper_outliers = len(df[cont_var][df[cont_var] > (np.quantile(df[cont_var],0.75) + 1.5*scipy.stats.iqr(df[cont_var]))])
        lower_outliers = len(df[cont_var][df[cont_var] < (np.quantile(df[cont_var],0.25) - 1.5*scipy.stats.iqr(df[cont_var]))])
        outlier_per = (upper_outliers + lower_outliers)/len(df[cont_var])
        df_outliers.loc[cont_var,'upper_outliers']=upper_outliers
        df_outliers.loc[cont_var,'lower_outliers']=lower_outliers
        df_outliers.loc[cont_var,'outlier_per']=round(outlier_per,2)*100
    return df_outliers

### Build model

In [25]:
def Average_Score_Model(model, X, y, size=0.25, cv=5):
    score_train=[]
    score_test=[]
    duration=[]
    ## Thực hiện lặp cv lần, mỗi lần lặp: tách X,y -> tính accuracy của train và test, time đưa vào danh sách
    for i in range(1,cv+1):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=size) 
        start = time.time()
        model.fit(X_train, y_train)
        end = time.time()
        score_train.append(model.score(X_train,y_train))
        score_test.append(model.score(X_test,y_test))
        duration.append((end-start)*1000)
    return np.mean(score_test),np.mean(duration) #trung bình score_train, score_test, time

In [26]:
def regression_model_evaluation(model_name, model,X_train,y_train,X_test,y_test):
    yhat_test = model.predict(X_test)
    # tinh R^2
    R2_train = model.score(X_train,y_train)
    R2_test = model.score(X_test,y_test)
    # tinh RMSE
    mse_test = mean_squared_error(y_true=y_test, y_pred=yhat_test)
    rmse_test = math.sqrt(mse_test)
    #result
    result = pd.DataFrame([[R2_train,R2_test,rmse_test]],
                          columns=['R2_train','R2_test','RMSE_test'],
                          index=[model_name])
    return result

In [27]:
def regression_model_evaluation2(model,X, y):
    yhat = model.predict(X)
    # tinh R^2
    R2_score = model.score(X, y)
    # tinh RMSE
    mse = mean_squared_error(y, yhat)
    rmse= math.sqrt(mse)
    #result
    result = pd.DataFrame([[R2_score,rmse]],
                          columns=['R2_score','rmse'],
                          index=['Result'])
    return result

In [28]:
# def cau_truc_func_1(lst_cau_truc):
#     new_lst_cau_truc=[]
#     for i in range(0, len(lst_cau_truc)): 
#         cau_truc = re.findall(r'\D+', lst_cau_truc[i])[0]
#         so_luong = re.findall(r'\d+', lst_cau_truc[i])
#         if len(so_luong)==0:
#             so_luong = 0
#         else:
#             so_luong=int(so_luong[0])
#         new_lst_cau_truc.append(cau_truc)
#         new_lst_cau_truc.append(so_luong)      
#     return new_lst_cau_truc

In [29]:
# def cau_truc_func_2(loai_cau_truc,new_lst_cau_truc):
#     if loai_cau_truc in new_lst_cau_truc:
#         if loai_cau_truc == 'cótầnghầm':
#             so_luong = 1
#         else:
#         #tên cấu trúc có trong new_lst_cau_truc
#             index=new_lst_cau_truc.index(loai_cau_truc)
#             so_luong=new_lst_cau_truc[index+1]
#         return so_luong
#     else:
#         #tên cấu trúc không có trong new_lst_cau_truc
#         return np.NaN